In [1]:
from splinter import Browser
from sys import platform
import os
import csv

import pandas as pd
import numpy as np

In [11]:
executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [4]:
filename = "nflcombinedata97to18.csv"
file_exists = os.path.isfile(filename)

with open(filename, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)
    
    if not file_exists:
        writer.writerow(["year","name","college","position","height_in","weight_lbs","40yards","benchpress","vert_leap_in","broad_jump_in","shuttle","3cone"])
        
    for x in range(1997,2019):
        browser.visit(f"http://nflcombineresults.com/nflcombinedata.php?year={x}")
        
        rowwww = browser.find_by_css('tr.tablefont')
        
        
        for seed in rowwww:
            writer.writerow([seed.find_by_css("td:nth-child(1)").text, \
                             seed.find_by_css("td:nth-child(2)").text, \
                             seed.find_by_css("td:nth-child(3)").text, \
                             seed.find_by_css("td:nth-child(4)").text, \
                             seed.find_by_css("td:nth-child(5)").text, \
                             seed.find_by_css("td:nth-child(6)").text, \
                             seed.find_by_css("td:nth-child(8)").text, \
                             seed.find_by_css("td:nth-child(9)").text, \
                             seed.find_by_css("td:nth-child(10)").text, \
                             seed.find_by_css("td:nth-child(11)").text, \
                             seed.find_by_css("td:nth-child(12)").text, \
                             seed.find_by_css("td:nth-child(13)").text])

In [6]:
df = pd.read_csv(filename)
df.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone
0,1997,Kevin Abrams,Syracuse,CB,68.4,174,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,Mike Adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51
2,1997,Winston Alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85
3,1997,Ink Aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60
4,1997,Eric Allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27


In [7]:
#remove punctuations and lower case the names
df["name"] = df["name"].str.lower()
df["name"] = df["name"].str.replace(".", "")
df["name"] = df["name"].str.replace("\'", "")
df["name"] = df["name"].str.replace("-", "")
df.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone
0,1997,kevin abrams,Syracuse,CB,68.4,174,NaN,NaN,NaN,NaN,NaN,NaN
1,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51
2,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85
3,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60
4,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27


In [8]:
#remove rows that have no measurements in any of the 6 events
combineonly_df = df[df["40yards"].notnull() | df["benchpress"].notnull() | df["vert_leap_in"].notnull() | df["broad_jump_in"].notnull() | df["shuttle"].notnull() | df["3cone"].notnull()]

In [10]:
#data that the names are lowercase and no puncutation marks 6749
combineonly_df.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone
1,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51
2,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85
3,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60
4,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27
5,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,NaN,NaN


In [12]:
filename = "drafted97to18.csv"
file_exists = os.path.isfile(filename)

with open(filename, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)
    
    if not file_exists:
        writer.writerow(["year","name","drafted"])
        
    for x in range(1997,2019):
        browser.visit(f"http://www.drafthistory.com/index.php/years/{x}")
                
        rowwww = browser.find_by_css('table tbody tr')
                
        for seed in rowwww:
            try:
                name = seed.find_by_css("td:nth-child(4)").text
                
                if (len(name) > 4):
                    writer.writerow([x,name,1])
            except AttributeError:
                pass

In [14]:
drafted = pd.read_csv(filename)
drafted.tail()

,year,name,drafted
5570,2018,Rod Taylor,1
5571,2018,Auden Tate,1
5572,2018,Korey Cunningham,1
5573,2018,Austin Proehl,1
5574,2018,Trey Quinn,1


In [16]:
drafted["name"] = drafted["name"].str.lower()
drafted["name"] = drafted["name"].str.replace(".", "")
drafted["name"] = drafted["name"].str.replace("\'", "")
drafted["name"] = drafted["name"].str.replace("-", "")
drafted.tail()

,year,name,drafted
5570,2018,rod taylor,1
5571,2018,auden tate,1
5572,2018,korey cunningham,1
5573,2018,austin proehl,1
5574,2018,trey quinn,1


In [18]:
#Confirm the 2015 data
combineonly_df.groupby("year")["name"].count()

year
1997    280
1998    269
1999    377
2000    258
2001    266
2002    270
2003    259
2004    274
2005    285
2006    279
2007    271
2008    267
2009    285
2010    287
2011    302
2012    326
2013    293
2014    301
2015    736
2016    303
2017    305
2018    256
Name: name, dtype: int64

So as we see, we have 2015 having 736 people, unlikely when all the years are about 200-400 players, we need to remove the extra rows

In [19]:
browser.visit(f"https://www.sbnation.com/nfl/2015/2/6/7993765/nfl-combine-2015-invite-list")

In [22]:
text2015 = browser.find_by_css("div.c-entry-content").text.lower()

In [27]:
counter = 0
for index, row in combineonly_df.iterrows():
    if (row["year"] == 2015):
        if (row["name"] not in text2015):
            combineonly_df = combineonly_df.drop([index])
            counter += 1
        
counter

451

In [28]:
len(combineonly_df)

6298

In [29]:
new_df = combineonly_df.merge(drafted, how="left", on=["name","year"])

In [31]:
new_df.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted
0,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51,NaN
1,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85,NaN
2,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,NaN
3,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27,NaN
4,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,NaN,NaN,NaN


In [33]:
new_df["drafted"] = new_df["drafted"].replace(np.nan,0)
new_df.head()
###----------------------------

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted
0,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51,0.0
1,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85,0.0
2,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,0.0
3,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27,0.0
4,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,NaN,NaN,0.0


In [34]:
new_df.groupby("year")["name"].count()

year
1997    280
1998    269
1999    377
2000    258
2001    266
2002    270
2003    259
2004    274
2005    287
2006    279
2007    271
2008    267
2009    285
2010    287
2011    302
2012    326
2013    293
2014    301
2015    285
2016    303
2017    305
2018    256
Name: name, dtype: int64

In [21]:
new_df = pd.to_numeric(new_df["drafted"], downcast="integer")

In [35]:
new_df['drafted'] = new_df['drafted'].astype(object)

In [37]:
new_df.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted
0,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51,0
1,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85,0
2,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,0
3,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27,0
4,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,NaN,NaN,0


In [38]:
cleaned = new_df

In [39]:
#save dataframe
cleaned.to_csv('cleaned_NFL_combine_data.csv', index=False)

In [41]:
cleaned.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted
0,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51,0
1,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85,0
2,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,0
3,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27,0
4,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,NaN,NaN,0


In [44]:
new_df["drafted"][new_df['drafted'] == 0] = "N"

C:\Users\dataviz\AppData\Local\Continuum\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [47]:
new_df.head()

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,shuttle,3cone,drafted
0,1997,mike adams,Texas,WR,71.4,184,4.59,NaN,29.5,112.0,4.38,7.51,N
1,1997,winston alderson,Arkansas,OT,76.1,344,5.60,NaN,24.5,92.0,5.25,8.85,N
2,1997,ink aleaga,Washington,ILB,73.0,227,4.87,NaN,31.0,107.0,4.33,7.60,N
3,1997,eric allen,Indiana,FS,72.0,205,4.78,NaN,34.0,116.0,4.22,7.27,N
4,1997,james allen,Oklahoma,RB,70.1,212,NaN,11.0,31.0,NaN,NaN,NaN,N


In [46]:
new_df.to_csv("cleaned_NFL_combine_97_to_2018.csv", index=False)